## makemore: becoming a backprop ninja

In [284]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [285]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [286]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [287]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [288]:
# ok biolerplate done, now we get to the action:

In [289]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad) # less strict than above: if some floating point downstream divergence, then this will consider some wiggle room
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [290]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [291]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [292]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n) -> Bessel's correction makes a biased estimate of variance unbiased (we are working with samples)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3498, grad_fn=<NegBackward0>)

In [293]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

# Note: dsomething = dloss/dsomething

# cmp('logprobs', dlogprobs, logprobs)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

1. dlogprobs - dloss/dlogprobs

In [294]:
# loss = -logprobs[range(n), Yb].mean()

# logprobs = [a, b, c]
# loss = -(a + b + c)/3
# dloss/da = loss/db = loss/dc = -1/3
# dlogprobs (dloss/dlogprobs) = [-1/3, -1/3, -1/3]

In [295]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


2. dprobs

In [296]:
# logprobs = probs.log()

# dprobs = dloss/dlogprobs * dlogprobs/dprobs

# dlogprobs/dprobs = 1/probs (derivative of logx wrt x)

In [297]:
dprobs = dlogprobs * (1.0/probs)

cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


3. dcounts_sum_inv

In [298]:
# probs = counts * counts_sum_inv

# dprobs/dcounts_sum_inv = counts

# dcounts_sum_inv = dloss/dlogprobs * dlogprobs/dprobs * dprobs/dcounts_sum_inv

In [299]:
dcounts_sum_inv = dprobs * counts
dcounts_sum_inv.shape

torch.Size([32, 27])

In [300]:
counts_sum_inv.shape

torch.Size([32, 1])

To make dcounts_sum_inv the same shape as counts_sum_inv, we take the sum across the second dimension (1). 

Note: Recall from tinygrad that if a node (neuron) is used multiple times, its gradients from all those branches sum up during backprop.

Note: (32, 27) * (32, 1) -> element-wise multiplication -> second tensor's second dim (=1) broadcasted 27 times for the 27 cols in the first tensor -> broadcasting is what reuses the element so their contributions need to be summed up -> this happens 32 times for 32 rows

1 element -> use 27 times -> 27 grads -> sum -> 1 gradient across 27 elements

In [301]:
dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True)

cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


4. dcounts_sum

In [302]:
# counts_sum_inv = counts_sum**-1

# dcounts_sum = dloss/dlogprobs * dlogprobs/dprobs * dprobs/dcounts_sum_inv * dcounts_sum_inv/dcounts_sum
# dcounts_sum = dcounts_sum_inv * dcounts_sum_inv/dcounts_sum

# dcounts_sum_inv/dcounts_sum = -counts_sum**-2

In [303]:
dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


Note that `probs = counts * counts_sum_inv` so we must calculate both dprobs/dcounts and dprobs/dcounts_sum_inv

In [304]:
# dprobs/dcounts = counts_sum_inv
# dloss/dprobs (dprobs) = dloss/dlogprobs * dlogprobs/dprobs * dprobs/dcounts
# dloss/dcounts = dloss/dprobs * dprobs/dcounts

dcounts = dprobs * counts_sum_inv

5. dcounts

In [305]:
# counts_sum = counts.sum(1, keepdims=True)

# dcounts = dcounts_sum * dcounts_sum/dcounts

In [306]:
counts.shape, counts_sum.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

Each row: count_sum([a, b, c]) -> (a + b + c) -> dcount_sum/dcounts -> [d(a + b + c)/da, d(a + b + c)/db, d(a + b + c)/dc] -> [1, 1, 1] 

In [307]:
# But dcounts also came from branch 4 so we need to sum up the gradients from both branches

dcounts += dcounts_sum * torch.ones_like(counts)

cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


6. dnorm_logits

In [308]:
# counts = norm_logits.exp()

# dcounts/dnorm_logits = norm_logits.exp() -> This is counts variable above

dnorm_logits = dcounts * counts

cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


7. dlogit_maxes

In [309]:
# norm_logits = logits - logit_maxes 

dlogits = dnorm_logits.clone() # logit_maxes is a constant during derivative wrt logits

In [310]:
norm_logits.shape, logit_maxes.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [311]:
# norm_logits = logits - logit_maxes
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# But logit_maxes = logits.max(1, keepdim=True).values
# We will account for this in the next step

cmp('logit_maxes', dlogit_maxes, logit_maxes)

logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0


Note one more thing. `logit_maxes` was added for numerical stability. You can verify that adding or subbing a number to logits keeps `probs` unchanged -> therefore, loss unchanged. In other words, loss is unaffected by logit_maxes and the derivative will be 0 (can be verified that `dlogit_maxes` close to 0)

In [312]:
dlogit_maxes

tensor([[ 4.6566e-09],
        [-9.3132e-10],
        [-3.7253e-09],
        [-9.3132e-10],
        [ 3.7253e-09],
        [ 1.6298e-09],
        [ 9.3132e-10],
        [ 1.8626e-09],
        [ 4.6566e-09],
        [ 3.2596e-09],
        [-5.1223e-09],
        [ 2.3283e-10],
        [-4.6566e-10],
        [-4.6566e-09],
        [-9.3132e-10],
        [ 3.0268e-09],
        [-4.1910e-09],
        [ 1.6298e-09],
        [-2.3283e-09],
        [-9.3132e-10],
        [ 4.1910e-09],
        [ 3.7253e-09],
        [-4.1910e-09],
        [-2.3283e-09],
        [-1.8626e-09],
        [-2.7940e-09],
        [ 4.6566e-10],
        [-1.8626e-09],
        [ 9.3132e-10],
        [ 0.0000e+00],
        [ 2.3283e-09],
        [-2.3283e-10]], grad_fn=<SumBackward1>)

8. logits

In [313]:
# logit_maxes = logits.max(1, keepdim=True).values

`logit_maxes` plucks out the max value from each row of logits. The derivative wrt logits will be 1 for the plucked out values and 0 for the others for each row.

In [314]:
dlogits += F.one_hot(logits.argmax(dim=1), num_classes=logits.shape[1]) * dlogit_maxes 

# If y = max(x₁, x₂, x₃) -> ∂loss/∂x₁ = ∂loss/∂y * ∂y/∂x₁
# ∂x₁ = ∂y/∂x₁ * ∂y

cmp('logits', dlogits, logits)

logits          | exact: True  | approximate: True  | maxdiff: 0.0


9. dh, dW2, db2

In [315]:
# logits = h @ W2 + b2 

In [316]:
dlogits.shape, h.shape, W2.shape, b2.shape

(torch.Size([32, 27]),
 torch.Size([32, 64]),
 torch.Size([64, 27]),
 torch.Size([27]))

These are matrix ops and follow matrix calculus. 

![image](images/Matrix_calc.png)
![image](images/Matrix_calc2.png)

In [317]:
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

cmp('dh', dh, h)
cmp('dW2', dW2, W2)
cmp('db2', db2, b2)

dh              | exact: True  | approximate: True  | maxdiff: 0.0
dW2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0


10.  dhpreact

In [318]:
# h = torch.tanh(hpreact)

dhpreact = dh * (1.0 - h**2)
cmp('dhpreact', dhpreact, hpreact)

dhpreact        | exact: True  | approximate: True  | maxdiff: 0.0


11. dbngain, dbnraw, dbnbias

In [319]:
# hpreact = bngain * bnraw + bnbias

In [320]:
hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [321]:
# y = x * b -> If y is (m, s) and x is (1, s), x is getting broadcasted across m rows and we need to sum up gradients across m rows
dbngain = (dhpreact * bnraw).sum(0, keepdim=True)

# y = x * b -> If y and x are the same shape, there was no broadcasting
dbnraw = bngain * dhpreact

# Again bnbias is (1, 64) and hpreact is (32, 64)
dbnbias = dhpreact.sum(0)

cmp('dbngain', dbngain, bngain)
cmp('dbnraw', dbnraw, bnraw)
cmp('dbnbias', dbnbias, bnbias)

dbngain         | exact: True  | approximate: True  | maxdiff: 0.0
dbnraw          | exact: True  | approximate: True  | maxdiff: 0.0
dbnbias         | exact: True  | approximate: True  | maxdiff: 0.0


12. dbnvar_inv

In [322]:
# bnraw = bndiff * bnvar_inv

In [323]:
bnraw.shape, bndiff.shape, bnvar_inv.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [324]:
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)

dbnvar_inv      | exact: True  | approximate: True  | maxdiff: 0.0


Also,

In [325]:
dbndiff = bnvar_inv * dbnraw

This will accumulate later as we add gradients from diff branches.

13. dbnvar

In [326]:
# bnvar_inv = (bnvar + 1e-5)**-0.5

dbnvar = dbnvar_inv * (-0.5*(bnvar + 1e-5)**-1.5)

cmp('dbnvar', dbnvar, bnvar)


dbnvar          | exact: True  | approximate: True  | maxdiff: 0.0


14. dbndiff2, dbndiff

In [327]:
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) 

In [328]:
bnvar.shape, bndiff2.shape

(torch.Size([1, 64]), torch.Size([32, 64]))

Say `bndiff2` [b1, b2, b3] = [1, 2, 3] -> `bnvar` = (1/2) * (b1 + b2 + b3) -> dbnvar/db1 = dbnvar/db2 = dbnvar/db3 = 1/2 -> dbnvar/dbndiff2 = [1/2, 1/2, 1/2]

In [329]:
# dbnvar/dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2)
dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar

dbndiff += 2* bndiff * dbndiff2

cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)

bndiff2         | exact: True  | approximate: True  | maxdiff: 0.0
bndiff          | exact: True  | approximate: True  | maxdiff: 0.0


15. dbnmeani, dhprebn

In [330]:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani

bndiff.shape, bnmeani.shape, hprebn.shape

(torch.Size([32, 64]), torch.Size([1, 64]), torch.Size([32, 64]))

In [331]:
dhprebn = dbndiff.clone() # dbndiff/dhprebn = tensor of ones

cmp('dhprebn', dhprebn, hprebn)
# One more branch coming for dhprebn

dhprebn         | exact: False | approximate: False | maxdiff: 0.0009581613121554255


In [332]:
# bndiff/dmeani = -1 but bnmeani is broadcasted as the mean for the 32 rows of bndiff
dbnmeani = (-dbndiff).sum(0)

cmp('dbnmeani', dbnmeani, bnmeani)

dbnmeani        | exact: True  | approximate: True  | maxdiff: 0.0


`dhprebn` is False cause of this step: `# bnmeani = 1/n*hprebn.sum(0, keepdim=True)`. We need to babckprop `dhprebn` to `dbnmeani`

In [333]:
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

cmp('dhprebn', dhprebn, hprebn)

dhprebn         | exact: True  | approximate: True  | maxdiff: 0.0


16. dembcat, dW1, db1

In [334]:
# hprebn = embcat @ W1 + b1

In [335]:
hprebn.shape, embcat.shape, W1.shape, b1.shape

(torch.Size([32, 64]),
 torch.Size([32, 30]),
 torch.Size([30, 64]),
 torch.Size([64]))

`b1` is broadcasted across `hprebn`'s first dimension (becomes (1, 64) first not (64, 1))

In [336]:
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, b1)

dembcat         | exact: True  | approximate: True  | maxdiff: 0.0
dW1             | exact: True  | approximate: True  | maxdiff: 0.0
db1             | exact: True  | approximate: True  | maxdiff: 0.0


17. demb

In [ ]:
# embcat = emb.view(emb.shape[0], -1)

In [337]:
embcat.shape, emb.shape

(torch.Size([32, 30]), torch.Size([32, 3, 10]))

View is just different interpretation of the same tensor. It isn't any specific operation that backprops.

In [338]:
demb = dembcat.view(emb.shape)

cmp('demb', demb, emb)

demb            | exact: True  | approximate: True  | maxdiff: 0.0


19. dC

In [ ]:
# emb = C[Xb]

In [339]:
emb.shape, C.shape, Xb.shape

(torch.Size([32, 3, 10]), torch.Size([27, 10]), torch.Size([32, 3]))

In [342]:
Xb[:5]

tensor([[ 1,  1,  4],
        [18, 14,  1],
        [11,  5,  9],
        [ 0,  0,  1],
        [12, 15, 14]])

In [343]:
C[[1, 1, 4]]

tensor([[-0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,  1.5618, -1.6261,
          0.6772, -0.8404],
        [-0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,  1.5618, -1.6261,
          0.6772, -0.8404],
        [-0.9648, -0.2321, -0.3476,  0.3324, -1.3263,  1.1224,  0.5964,  0.4585,
          0.0540, -1.7400]], grad_fn=<IndexBackward0>)

1. If `Xb[i,j] = k`, then `emb[i,j`] is a copy of `C[k]`
2. Therefore, `∂emb[i,j]/∂C[k] = 1` (identity matrix)
3. By chain rule: `∂loss/∂C[k] = ∂loss/∂emb[i,j] * 1 = demb[i,j]`

In [340]:
dC = torch.zeros_like(C)

for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i, j] # 1 then 1 then 4 (first row and so on)
        dC[ix] += demb[i, j] # 10D 1 then 10D 1 then 10D 4 (We add when gradients are of the same element like 1)

cmp('dC', dC, C)

dC              | exact: True  | approximate: True  | maxdiff: 0.0


For row 0 [1, 1, 4]:

i=0, j=0: ix=1 → dC[1] += demb[0,0] (first gradient for index 1)

i=0, j=1: ix=1 → dC[1] += demb[0,1] (second gradient for index 1, gets added to first)

In [341]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

dprobs = dlogprobs * (1.0/probs)

dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True)

dcounts = dprobs * counts_sum_inv
dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

dcounts += dcounts_sum * torch.ones_like(counts)

dnorm_logits = dcounts * counts
dlogits = dnorm_logits.clone()

dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

dhpreact = dh * (1.0 - h**2)

dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw

dbnvar = dbnvar_inv * (-0.5*(bnvar + 1e-5)**-1.5)

dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar
dbndiff += 2* bndiff * dbndiff2

dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i, j]
        dC[ix] += demb[i, j]


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [44]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.362204074859619 diff: 2.384185791015625e-07


In [45]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = None # TODO. my solution is 3 lines
# -----------------

#cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

In [46]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(7.1526e-07, grad_fn=<MaxBackward1>)


In [47]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
# dhprebn = None # TODO. my solution is 1 (long) line
# -----------------

# cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [48]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  # YOUR CODE HERE :)
  dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    #p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  if i >= 100: # TODO: delete early breaking when you're ready to train the full net
    break

12297
      0/ 200000: 3.7573


In [49]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [50]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [51]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.6684978008270264
val 2.667092800140381


In [52]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [53]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

carmahxqto.
hqvifkigrixreetl.
hklansaeja.
hntnfapliahc.
kaqhi.
oremari.
cemiiv.
kklegg.
hhlm.
eoi.
dqsnnn.
shlin.
a.
vadiq.
wane.
ogdiarixi.
fkcekphrran.
ea.
ecoia.
gtlefay.
